In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from keras.models import Sequential
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Embedding
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from keras import Model
from keras.layers import Flatten, Input, Layer, GlobalMaxPooling1D, LSTM, Bidirectional, Concatenate

# Testing

In [ ]:
# count number of characters
test['char_len'] = test.text.str.len()

# count number of words
word_tokens = [len(word_tokenize(tweet)) for tweet in test.text]
test['word_len'] = word_tokens

# count number of sentence
sent_tokens = [len(sent_tokenize(tweet)) for tweet in test.text]
test['sent_len'] = sent_tokens

In [ ]:
# polarity and subjectivity
test['polarity'] = [TextBlob(tweet).sentiment.polarity for tweet in test.text]
test['subjectivity'] = [TextBlob(tweet).sentiment.subjectivity for tweet in test.text]

# exclaimation and question marks
test['exclaimation_num'] = [tweet.count('!') for tweet in test.text]
test['questionmark_num'] = [tweet.count('?') for tweet in test.text]

# count number of hashtags and mentions
def count_url_hashtag_mention(text):
    urls_num = len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text))
    word_tokens = text.split()
    hash_num = len([word for word in word_tokens if word[0] == '#' and word.count('#') == 1]) # only appears once in front of word 
    mention_num = len([word for word in word_tokens if word[0] == '@' and word.count('@') == 1]) # only appears once in front of word 
    return urls_num, hash_num, mention_num

url_num, hash_num, mention_num = zip(*[count_url_hashtag_mention(tweet) for tweet in test.text])
test = test.assign(url_num = url_num, hash_num = hash_num, mention_num = mention_num)

# count number of contractions
contractions = ["'t", "'re", "'s", "'d", "'ll", "'ve", "'m"]
test['contraction_num'] = [sum([tweet.count(cont) for cont in contractions]) for tweet in test.text]

In [ ]:
## Replace NaNs with 'None'
test.keyword.fillna('None', inplace=True) 

## Expand Contractions
def decontraction(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

test.text = [decontraction(tweet) for tweet in test.text]

## Remove Emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

print(remove_emoji("OMG there is a volcano eruption!!! 😭😱😷"))

test.text = test.text.apply(lambda x: remove_emoji(x))

In [ ]:
## Remove URLs
test.text = test.text.apply(lambda x: remove_url(x))

## Remove Punctuations except '!?'
def remove_punct(text):
    new_punct = re.sub('\ |\!|\?', '', punctuation)
    table=str.maketrans('','',new_punct)
    return text.translate(table)
test.text = test.text.apply(lambda x: remove_punct(x))

## Replace amp
def replace_amp(text):
    text = re.sub(r" amp ", " and ", text)
    return text

test.text = test.text.apply(lambda x: replace_amp(x))

In [ ]:
# from wordsegment import load, segment
# load()

# test.text = test.text.apply(lambda x: ' '.join(segment(x)))

#test = pd.read_csv('')

In [ ]:
## Lemmatization

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemma(text):
    words = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w.lower(), pos='v') for w in words])

test.text = test.text.apply(lambda x: lemma(x))

In [ ]:
# tokenize
test_text = test['text']
test_text = tokenizer_1.texts_to_sequences(test_text)

# padding
test_text = pad_sequences(test_text, padding='post', maxlen=50)

print('X_test shape:', test_text.shape)

In [ ]:
# lstm prediction
# model.predict(test_text)
lstm_model.load_weights('lstm_model.keras')
submission = test.copy()[['id']]
submission['target'] = lstm_model.predict_classes(test_text)
submission.to_csv('submission.csv', index=False)
display(submission.head())

In [ ]:
# bi-lstm attention prediction
model_attn.load_weights('attn_model.keras')
submission_attn = test.copy()[['id']]
submission_attn['target'] = model_attn.predict(test_text)
submission_attn['target'] = submission_attn['target'].apply(lambda x: 1 if x >=0.5 else 0)
submission_attn.to_csv('submission_attn.csv', index=False)
display(submission_attn.head())